In [2]:
### Import packages for data manipulation

import pandas as pd
import numpy as np
import re

### Import packages to create absolute file path & make code independent of operating system

from pathlib import Path
import os.path

import warnings
warnings.filterwarnings("ignore")

### Import packages for feature extraction

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import accuracy_score, classification_report
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB

In [3]:
### Read in dataset

base_path = Path("__file__").parent

full_path = (base_path / "../../data/processed/stackoverflow_preprocessed.csv").resolve()

stackoverflow = pd.read_csv(os.path.join(full_path))

In [3]:
stackoverflow = pd.read_csv("/Users/HenriekeMax/Documents/Career_Development/GitHub/FrauenLoop_NLP_Project_2020/data/processed/stackoverflow_preprocessed.csv")

In [4]:
stackoverflow.head()

,Unnamed: 0,score,question_title,question_text,answer_count,comment_count,creation_date,favorite_count,tags,view_count,answer_text,score_cat,question_title_clean,question_text_clean,tags_clean,answer_text_clean
0,0,0,How to avoid Bot repeating command in groups?,<p>I created a new Telegram Bot which maintain...,2,0,2019-08-23 14:05:29.463000+00:00,0.0,telegram-bot|python-telegram-bot,79,"<p>The bot only is replying message, not creat...",bad,avoid bot repeat command group,create new telegram bot maintain simple list c...,telegrambot pythontelegrambot,bot reply message create message use sendmessage
1,1,-3,how to host Asp.net core web application..?,<p>I have more confusion with hosting my appli...,2,0,2020-04-27 22:38:03.737000+00:00,NaN,asp.net-core|shared-hosting|cloud-hosting,47,<p>If choose their windows cloud hosting platf...,bad,host asp net core web application,confusion host application build application a...,asp netcore sharedhosting cloudhosting,choose window cloud host platform net core wor...
2,2,0,discord.ext.commands.errors.MissingRequiredArg...,<p>i would like make a command for set permiss...,2,0,2020-02-21 01:10:53.553000+00:00,NaN,discord.py|discord.py-rewrite,103,"<p>What you're doing right now, is <strong>req...",bad,discord ext command error missingrequiredargum...,would like make command set permission text ch...,discord py discord pyrewrite,right now require guild argument command bot a...
3,3,0,JPA not saving to DB on WildFly 16,<p>After migrating from Wildfly-8.2.0 to Wildf...,1,1,2019-11-03 23:51:01.683000+00:00,NaN,jpa|jakarta-ee|persistence|jpa-2.1|wildfly-16,22,"<p>What was missing is to add the ""eclipselink...",bad,jpa save db wildfly,migrate wildfly wildfly jee application launch...,jpa jakartaee persistence jpa wildfly,miss add eclipselink jar file wildfly module s...
4,4,0,Conditionally rendering an array within an arr...,<p>I've tried many of the methods outlined by ...,1,5,2020-02-23 03:24:42.360000+00:00,NaN,javascript|html|reactjs,40,<p>You need to trigger a re-render. This is do...,bad,conditionally render array within array within...,ive try many method outline user dice im sure ...,javascript html reactjs,need trigger rerender do call tell content com...


In [5]:
### Define function to count 

def wordcounter(x):
    x = len(re.findall(r'\w+', x))
    return x

In [6]:
### Feature extraction

stackoverflow['answer_wordcount'] = stackoverflow['answer_text_clean'].apply(wordcounter)

TypeError: expected string or bytes-like object

In [7]:
vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform(stackoverflow['answer_text_clean'].apply(lambda x: np.str_(x)))

feature_names = vectorizer.get_feature_names()

X = X.toarray()

In [8]:
X.shape

(30000, 138096)

In [9]:
y = stackoverflow['score_cat']

In [10]:
y.shape

(30000,)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1)

model = BernoulliNB()
model.fit(X_train, y_train)
y_predict = model.predict(X_test)

print(accuracy_score(y_test, y_predict))
print(classification_report(y_test, y_predict))
pd.crosstab(y_test, y_predict)